In [0]:
REDSHIFT_USER = ''
REDSHIFT_PASSWORD = ''
REDSHIFT_HOST = ''
REDSHIFT_PORT = 5439
REDSHIFT_DATABASE = ''
REDSHIFT_SCHEMA = ''
IAM_ROLE = ''

TRANSFORMED_MOUNT = '/mnt/transformed' # path to mount serving bucket

In [0]:
def load_s3_to_redshift(type):
    print(f'loading {type} to redshift')

    conn = psycopg2.connect(dbname=REDSHIFT_DATABASE, host=REDSHIFT_HOST, port=REDSHIFT_PORT, user=REDSHIFT_USER, password=REDSHIFT_PASSWORD)
    cur = conn.cursor()

    for file in dbutils.fs.ls(f'{TRANSFORMED_MOUNT}/{type}/'):
        file_path = file.path
        if file_path.endswith('.parquet'):
            file_name = file_path.split('/')[-1]
            COPY_COMMAND = f"""
                            COPY {REDSHIFT_SCHEMA}.{type} FROM '{SERVING_BUCKET}/{type}/{file_name}'
                            IAM_ROLE '{IAM_ROLE}'
                            FORMAT AS PARQUET
                        """
            cur.execute(COPY_COMMAND)
            cur.execute('COMMIT;')
    print(f'done loading {type}')
            

In [0]:
load_s3_to_redshift('artists')
load_s3_to_redshift('albums')
load_s3_to_redshift('tracks')